# Data Engineer Certification Sample Practical Exam

HappyPaws, creates fun and educational apps for pet owners. 

HappyPaws wants to help pet owners understand their pets better by tracking their activities and health through the app. 

The data engineering team is responsible for making sure all the pet data from thousands of users is organized and safe, so pet owners can get tips to keep their pets happy and healthy.



## Task

HappyPaws has collected three datasets over the past year: 
 - "pet_activities.csv" which logs daily activities of pets, 
 - "pet_health.csv" which records vet visits and health issues, and 
 - "users.csv" which contains information about the pet owners. 

Each dataset contains unique identifiers for pets and/or their owners. 

The engineers developing the app currently write code to cross reference all of these data sources. 

They want to make things easier by having a single table with all data included.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset. 

The final dataset should provide a comprehensive view of each pet's activities, health records, and owner information. 

- To test your code, your manager will run only the code `all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')`
- Your `all_pet_data()` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.

## Data

The data that has been provided has the following structure

![image](image.png)

The function that you write must return data as described below. There should be a unique row for each activity/health visit. 

Where missing values are permitted, they should be in the default Python format.

|Column Name | Description |
|-------|-------|
| pet_id | Unique identifier for each pet. There should not be any missing values.|
| date | The date of the activity recorded or the date of the health visit, in date format. There should not be any missing values.|
| activity_type | The type of activity, one of 'Walking', 'Playing', 'Resting' or for rows that relate to a health visit, the value 'Health'. Missing values are permitted.|
| duration_minutes | The duration of the activity in minutes. For rows that relate to health visits, this should be 0. Missing values for other activities are permitted.|
| issue | The health issue identified or check-up note. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| resolution | The outcome or advice given for the issue. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| owner_id | Unique identifier for the pet owner. All pets must have an owner.|
| owner_age_group | The age group of the owner (e.g., 18-25, 26-35, etc.). Missing values are permitted.|
| pet_type | The type of pet (e.g., Dog, Cat). Missing values are permitted.|

In [10]:
# Use as many python cells as you wish to write your code

In [1]:
import pandas as pd

In [2]:
pet_activities = pd.read_csv("pet_activities.csv")
pet_activities.head()

,pet_id,date,activity_type,duration_minutes
0,9930,2023-08-01,Resting,-
1,3040,2023-07-24,Resting,4
2,2381,2023-01-11,Walking,24
3,3936,2022-09-15,Walking,73
4,8989,2023-02-06,Playing,77


In [3]:
pet_health = pd.read_csv("pet_health.csv")
pet_health.head()

,pet_id,visit_date,issue,resolution
0,9692,2022-10-09,Dental Cleaning,"Cleaning done, no dental issues"
1,9186,2022-05-15,Ear Infection,Medication and follow-up in 2 weeks for infection
2,2709,2023-12-24,Annual Checkup,"All clear, healthy diet recommended"
3,2513,2023-03-18,Annual Checkup,"Vaccination completed, next due in 1 year"
4,2016,2023-10-03,Dental Cleaning,"Cleaning done, no dental issues"


In [4]:
users = pd.read_csv("users.csv")
users.head()

,owner_id,pet_id,owner_age_group,pet_type
0,7016,9930,18-25,Dog
1,5637,3040,18-25,Rabbit
2,5366,2381,18-25,Rabbit
3,7063,3936,26-35,Hamster
4,3803,8989,36-45,Dog


In [5]:
def pet_activities_data(pet_activities_df, users_df):
    df = pet_activities_df.merge(users_df, on='pet_id')

    df["issue"] = None
    df["resolution"] = None

    df = df.loc[:, ["pet_id", "date", "activity_type", "duration_minutes", "issue", "resolution", "owner_id", "owner_age_group", "pet_type"]]

    return df

def pet_health_data(pet_health_df, users_df):
    df = pet_health_df.merge(users_df, on='pet_id')

    df["activity_type"] = "Health"
    df["duration_minutes"] = 0

    df = df.rename(columns={
        "visit_date": "date"
    })

    df = df.loc[:, ["pet_id", "date", "activity_type", "duration_minutes", "issue", "resolution", "owner_id", "owner_age_group", "pet_type"]]

    return df

def all_pet_data(pet_activities_filename, pet_health_filename, users_filename):
    pet_activities_df = pd.read_csv(pet_activities_filename)
    pet_health_df = pd.read_csv(pet_health_filename)
    users_df = pd.read_csv(users_filename)

    pet_activities_data_df = pet_activities_data(pet_activities_df, users_df)
    pet_health_data_df = pet_health_data(pet_health_df, users_df)

    all_data = pd.concat([pet_activities_data_df, pet_health_data_df], axis=0)

    all_data = all_data[all_data["duration_minutes"] != "-"]

    all_data['date'] = pd.to_datetime(all_data['date'])
    all_data['activity_type'] = all_data['activity_type'].astype('category')
    all_data['duration_minutes'] = all_data['duration_minutes'].astype('int')
    all_data['owner_age_group'] = all_data['owner_age_group'].astype('category')
    all_data['pet_type'] = all_data['pet_type'].astype('category')

    return all_data

In [6]:
df = all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')

In [7]:
print(df.dtypes)

pet_id                       int64
date                datetime64[ns]
activity_type             category
duration_minutes             int64
issue                       object
resolution                  object
owner_id                     int64
owner_age_group           category
pet_type                  category
dtype: object


In [8]:
print(df.isna().sum())

pet_id                0
date                  0
activity_type         0
duration_minutes      0
issue               751
resolution          751
owner_id              0
owner_age_group       0
pet_type              0
dtype: int64


In [9]:
print(df)

     pet_id       date activity_type  duration_minutes           issue  \
1      3040 2023-07-24       Resting                 4            None   
2      2381 2023-01-11       Walking                24            None   
3      3936 2022-09-15       Walking                73            None   
4      8989 2023-02-06       Playing                77            None   
5       627 2023-02-15       Resting                45            None   
..      ...        ...           ...               ...             ...   
935    9136 2022-04-15        Health                 0  Annual Checkup   
936    3228 2023-07-30        Health                 0          Injury   
937    9163 2023-01-20        Health                 0  Annual Checkup   
938    2805 2022-05-28        Health                 0          Injury   
939    8070 2022-09-14        Health                 0          Injury   

                                            resolution  owner_id  \
1                                          